In [1]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as LR

import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.ToTensor()


trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

batch_size = 200

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

print('==>>> total training batch number: {}'.format(len(trainloader)))
print('==>>> total testing batch number: {}'.format(len(testloader)))

# for i, data in enumerate(trainloader, 0):
#     print(len(data))

==>>> total training batch number: 300
==>>> total testing batch number: 50


In [3]:
# def show_batch(batch):
#     im = torchvision.utils.make_grid(batch)
#     plt.imshow(np.transpose(im.numpy(), (1, 2, 0)))
#     plt.show()


# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# print('Labels: ', labels)
# print('Batch shape: ', images.size())
# show_batch(images)

# images.view(batch_size, -1).size()

In [4]:
INPUTSIZE = 28*28
NBLAYERS = 9
LAYERSIZE = 25

class BN(nn.Module):
    def __init__(self, layersize):
        super(BN, self).__init__()
        self.gain = nn.Parameter(torch.ones(layersize))
        self.bias = nn.Parameter(torch.zeros(layersize))
        
    def forward(self, x):
        
        if (self.testing is False):
            mu = x.mean(1, keepdim=True)
            sigma = ((x-mu)**2).mean(1, keepdim=True).sqrt()
            
            # Normalize
            nx = (x-mu)/sigma
            
            # Adjust using learned parameters
            o = self.gain*nx + self.bias
            
        else:
            raise NotImplementedError("Testing mode batch normalization has not been implemented")


class IP(nn.Module):
    def __init__(self, layersize):
        super(IP, self).__init__()
        self.gain = nn.Parameter(torch.ones(layersize))
        self.bias = nn.Parameter(torch.zeros(layersize))
        
        self.alpha = nn.Buffer(torch.ones(layersize))
        self.beta = nn.Buffer(torch.zeros(layersize))
        
    def forward(self, x):

        # Normalize
        nx = (x-self.beta)/self.alpha

        # Adjust using learned parameters
        o = self.gain*nx + self.bias
        
    def update(self, u, v):
        
        Eu
        
                  

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, layer1size)
        self.fc2 = nn.Linear(layer1size, layer2size)
        self.fc3 = nn.Linear(layer2size, layer3size)
        
#         self.ILR = nn.Parameter(torch.tensor(0.001)) # Unnecessary?
        
        self.gain1 = nn.Parameter(torch.ones(layer1size))
        self.gain2 = nn.Parameter(torch.ones(layer2size))
        self.gain3 = nn.Parameter(torch.ones(layer3size))
        
        self.bias1 = nn.Parameter(torch.zeros(layer1size))
        self.bias2 = nn.Parameter(torch.zeros(layer2size))
        self.bias3 = nn.Parameter(torch.zeros(layer3size))

    def forward(self, x):
        u1 = self.fc1(x.view(batch_size, -1))
        y1 = F.tanh(self.gain1.data * u1 + self.bias1.data)
        u2 = self.fc2(y1)
        y2 = F.tanh(self.gain2.data * u2 + self.bias2.data)
        u3 = self.fc3(y2)
        y3 = F.relu(self.gain3.data * u3 + self.bias3.data)
        return u1, y1, u2, y2, u3, y3
    
dlayersize = 100
    
class DNet(nn.Module):
    def __init__(self, IP=False):
        super(DNet, self).__init__()

        self.fc1 = nn.Linear(28*28, dlayersize)
        self.fc2 = nn.Linear(dlayersize, dlayersize)
        self.fc3 = nn.Linear(dlayersize, dlayersize)
        self.fc4 = nn.Linear(dlayersize, dlayersize)
        self.fc5 = nn.Linear(dlayersize, dlayersize)
        self.fc6 = nn.Linear(dlayersize, dlayersize)
        self.fc7 = nn.Linear(dlayersize, dlayersize)
        self.fc8 = nn.Linear(dlayersize, dlayersize)
        self.fc9 = nn.Linear(dlayersize, 10)
        
        self.gain1 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
        self.gain2 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
        self.gain3 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
        self.gain4 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
        self.gain5 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
        self.gain6 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
        self.gain7 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
        self.gain8 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
        self.gain9 = nn.Parameter(torch.ones(10), requires_grad=False)
        
        self.bias1 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
        self.bias2 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
        self.bias3 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
        self.bias4 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
        self.bias5 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
        self.bias6 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
        self.bias7 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
        self.bias8 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
        self.bias9 = nn.Parameter(torch.zeros(10), requires_grad=False)
        
        if (IP == True):
            self.alpha1 = nn.Parameter(torch.ones(dlayersize), requires_grad=True)
            self.alpha2 = nn.Parameter(torch.ones(dlayersize), requires_grad=True)
            self.alpha3 = nn.Parameter(torch.ones(dlayersize), requires_grad=True)
            self.alpha4 = nn.Parameter(torch.ones(dlayersize), requires_grad=True)
            self.alpha5 = nn.Parameter(torch.ones(dlayersize), requires_grad=True)
            self.alpha6 = nn.Parameter(torch.ones(dlayersize), requires_grad=True)
            self.alpha7 = nn.Parameter(torch.ones(dlayersize), requires_grad=True)
            self.alpha8 = nn.Parameter(torch.ones(dlayersize), requires_grad=True)
            self.alpha9 = nn.Parameter(torch.ones(10), requires_grad=True)

            self.beta1 = nn.Parameter(torch.zeros(dlayersize), requires_grad=True)
            self.beta2 = nn.Parameter(torch.zeros(dlayersize), requires_grad=True)
            self.beta3 = nn.Parameter(torch.zeros(dlayersize), requires_grad=True)
            self.beta4 = nn.Parameter(torch.zeros(dlayersize), requires_grad=True)
            self.beta5 = nn.Parameter(torch.zeros(dlayersize), requires_grad=True)
            self.beta6 = nn.Parameter(torch.zeros(dlayersize), requires_grad=True)
            self.beta7 = nn.Parameter(torch.zeros(dlayersize), requires_grad=True)
            self.beta8 = nn.Parameter(torch.zeros(dlayersize), requires_grad=True)
            self.beta9 = nn.Parameter(torch.zeros(10), requires_grad=True)
            
        if (IP == False):
            self.alpha1 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
            self.alpha2 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
            self.alpha3 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
            self.alpha4 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
            self.alpha5 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
            self.alpha6 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
            self.alpha7 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
            self.alpha8 = nn.Parameter(torch.ones(dlayersize), requires_grad=False)
            self.alpha9 = nn.Parameter(torch.ones(10), requires_grad=False)

            self.beta1 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
            self.beta2 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
            self.beta3 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
            self.beta4 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
            self.beta5 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
            self.beta6 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
            self.beta7 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
            self.beta8 = nn.Parameter(torch.zeros(dlayersize), requires_grad=False)
            self.beta9 = nn.Parameter(torch.zeros(10), requires_grad=False)

    def forward(self, x):
        u1 = self.fc1(x.view(batch_size, -1))
        y1 = F.tanh(self.alpha1.data*(1/self.gain1.data * (u1 - self.bias1.data)) + self.beta1.data)
        u2 = self.fc2(y1)
        y2 = F.tanh(self.alpha2.data*(1/self.gain2.data * (u2 - self.bias2.data)) + self.beta2.data)
        u3 = self.fc3(y2)
        y3 = F.relu(self.alpha3.data*(1/self.gain3.data * (u3 - self.bias3.data)) + self.beta3.data)
        u4 = self.fc4(y3)
        y4 = F.relu(self.alpha4.data*(1/self.gain4.data * (u4 - self.bias4.data)) + self.beta4.data)
        u5 = self.fc5(y4)
        y5 = F.relu(self.alpha5.data*(1/self.gain5.data * (u5 - self.bias5.data)) + self.beta5.data)
        u6 = self.fc6(y5)
        y6 = F.relu(self.alpha6.data*(1/self.gain6.data * (u6 - self.bias6.data)) + self.beta6.data)
        u7 = self.fc7(y6)
        y7 = F.relu(self.alpha7.data*(1/self.gain7.data * (u7 - self.bias7.data)) + self.beta7.data)
        u8 = self.fc8(y7)
        y8 = F.relu(self.alpha8.data*(1/self.gain8.data * (u8 - self.bias8.data)) + self.beta8.data)
        u9 = self.fc9(y8)
        y9 = F.relu(self.alpha9.data*(1/self.gain9.data * (u9 - self.bias9.data)) + self.beta9.data)
        return u1, y1, u2, y2, u3, y3, u4, y4, u5, y5, u6, y6, u7, y7, u8, y8, u9, y9
    
criterion = nn.CrossEntropyLoss()
    
def train_deep_model(network, optimization, seed, IP=True):
    torch.manual_seed(seed)
#     print("Training", str(network))
    lambda1 = lambda epoch: epoch
    lambda2 = lambda epoch: 0.996 ** epoch
    eta = 0.003
    scheduler = LR.LambdaLR(optimization, lr_lambda=lambda2)
    for epoch in range(30):  # loop over the dataset multiple times
        scheduler.step()
        if(IP):
            Eu1 = torch.zeros(dlayersize).to(device)
            Ey1 = torch.zeros(dlayersize).to(device)
            Euy1 = torch.zeros(dlayersize).to(device)
            Eu2 = torch.zeros(dlayersize).to(device)
            Ey2 = torch.zeros(dlayersize).to(device)
            Euy2 = torch.zeros(dlayersize).to(device)
            Eu3 = torch.zeros(dlayersize).to(device)
            Ey3 = torch.zeros(dlayersize).to(device)
            Euy3 = torch.zeros(dlayersize).to(device)
            Eu4 = torch.zeros(dlayersize).to(device)
            Ey4 = torch.zeros(dlayersize).to(device)
            Euy4 = torch.zeros(dlayersize).to(device)
            Eu5 = torch.zeros(dlayersize).to(device)
            Ey5 = torch.zeros(dlayersize).to(device)
            Euy5 = torch.zeros(dlayersize).to(device)
            Eu6 = torch.zeros(dlayersize).to(device)
            Ey6 = torch.zeros(dlayersize).to(device)
            Euy6 = torch.zeros(dlayersize).to(device)
            Eu7 = torch.zeros(dlayersize).to(device)
            Ey7 = torch.zeros(dlayersize).to(device)
            Euy7 = torch.zeros(dlayersize).to(device)
            Eu8 = torch.zeros(dlayersize).to(device)
            Ey8 = torch.zeros(dlayersize).to(device)
            Euy8 = torch.zeros(dlayersize).to(device)
            Eu9 = torch.zeros(10).to(device)
            Ey9 = torch.zeros(10).to(device)
            Euy9 = torch.zeros(10).to(device)
            
            Euu1 = torch.zeros(dlayersize).to(device)
            Euu2 = torch.zeros(dlayersize).to(device)
            Euu3 = torch.zeros(dlayersize).to(device)
            Euu4 = torch.zeros(dlayersize).to(device)
            Euu5 = torch.zeros(dlayersize).to(device)
            Euu6 = torch.zeros(dlayersize).to(device)
            Euu7 = torch.zeros(dlayersize).to(device)
            Euu8 = torch.zeros(dlayersize).to(device)
            Euu9 = torch.zeros(10).to(device)
            
            Eyy1 = torch.zeros(dlayersize).to(device)
            Eyy2 = torch.zeros(dlayersize).to(device)
            Eyy3 = torch.zeros(dlayersize).to(device)
            Eyy4 = torch.zeros(dlayersize).to(device)
            Eyy5 = torch.zeros(dlayersize).to(device)
            Eyy6 = torch.zeros(dlayersize).to(device)
            Eyy7 = torch.zeros(dlayersize).to(device)
            Eyy8 = torch.zeros(dlayersize).to(device)
            Eyy9 = torch.zeros(10).to(device)
            
            iter_num = 0

        running_loss = 0.0

        for i, data in enumerate(trainloader, 0):

            if(IP):
                iter_num = iter_num+1

            # get the inputs
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimization.zero_grad()

            # forward + backward + optimize
            u1, y1, u2, y2, u3, y3, u4, y4, u5, y5, u6, y6, u7, y7, u8, y8, u9, y9 = network(inputs)
            loss = criterion(y9, labels)
            loss.backward()
            optimization.step()
            
            if(IP):
                Eu1 = Eu1 + u1/batch_size
                Ey1 = Ey1 + y1/batch_size
                Euy1 = Euy1 + u1*y1/batch_size
                Eu2 = Eu2 + u2/batch_size
                Ey2 = Ey2 + y2/batch_size
                Euy2 = Euy2 + u2*y2/batch_size
                Eu3 = Eu3 + u3/batch_size
                Ey3 = Ey3 + y3/batch_size
                Euy3 = Euy3 + u3*y3/batch_size
                Eu4 = Eu4 + u4/batch_size
                Ey4 = Ey4 + y4/batch_size
                Euy4 = Euy4 + u4*y4/batch_size
                Eu5 = Eu5 + u5/batch_size
                Ey5 = Ey5 + y5/batch_size
                Euy5 = Euy5 + u5*y5/batch_size
                Eu6 = Eu6 + u6/batch_size
                Ey6 = Ey6 + y6/batch_size
                Euy6 = Euy6 + u6*y6/batch_size
                Eu7 = Eu7 + u7/batch_size
                Ey7 = Ey7 + y7/batch_size
                Euy7 = Euy7 + u7*y7/batch_size
                Eu8 = Eu8 + u8/batch_size
                Ey8 = Ey8 + y8/batch_size
                Euy8 = Euy8 + u8*y8/batch_size
                Eu9 = Eu9 + u9/batch_size
                Ey9 = Ey9 + y9/batch_size
                Euy9 = Euy9 + u9*y9/batch_size
                
                Euu1 = Euu1 + u1*u1/batch_size
                Euu2 = Euu2 + u2*u2/batch_size
                Euu3 = Euu3 + u3*u3/batch_size
                Euu4 = Euu4 + u4*u4/batch_size
                Euu5 = Euu5 + u5*u5/batch_size
                Euu6 = Euu6 + u6*u6/batch_size
                Euu7 = Euu7 + u7*u7/batch_size
                Euu8 = Euu8 + u8*u8/batch_size
                Euu9 = Euu9 + u9*u9/batch_size
                
                Eyy1 = Eyy1 + y1*y1/batch_size
                Eyy2 = Eyy2 + y2*y2/batch_size
                Eyy3 = Eyy3 + y3*y3/batch_size
                Eyy4 = Eyy4 + y4*y4/batch_size
                Eyy5 = Eyy5 + y5*y5/batch_size
                Eyy6 = Eyy6 + y6*y6/batch_size
                Eyy7 = Eyy7 + y7*y7/batch_size
                Eyy8 = Eyy8 + y8*y8/batch_size
                Eyy9 = Eyy9 + y9*y9/batch_size

            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

        if(IP):
#             # Original
#             network.gain1.data = network.gain1.data + eta * (1/network.gain1.data - 2*Euy1/iter_num - 0.5*network.gain1.data)
#             network.gain2.data = network.gain2.data + eta * (1/network.gain2.data - 2*Euy2/iter_num - 0.5*network.gain2.data)
#             network.gain3.data = network.gain3.data + eta * (1/network.gain3.data - 2*Euy3/iter_num - 0.5*network.gain3.data)
#             network.gain4.data = network.gain4.data + eta * (1/network.gain4.data - 2*Euy4/iter_num - 0.5*network.gain4.data)
#             network.gain5.data = network.gain5.data + eta * (1/network.gain5.data - 2*Euy5/iter_num - 0.5*network.gain5.data)
#             network.gain6.data = network.gain6.data + eta * (1/network.gain6.data - 2*Euy6/iter_num - 0.5*network.gain6.data)
#             network.gain7.data = network.gain7.data + eta * (1/network.gain7.data - 2*Euy7/iter_num - 0.5*network.gain7.data)
#             network.gain8.data = network.gain8.data + eta * (1/network.gain8.data - 2*Euy8/iter_num - 0.5*network.gain8.data)
#             network.gain9.data = network.gain9.data + eta * (1/network.gain9.data - 2*Euy9/iter_num - 0.5*network.gain9.data)
            
#             # Covariance
#             network.gain1.data = network.gain1.data + eta * (1/network.gain1.data - 2*(Euy1/iter_num - Eu1*Ey1/iter_num))
#             network.gain2.data = network.gain2.data + eta * (1/network.gain2.data - 2*(Euy2/iter_num - Eu2*Ey2/iter_num))
#             network.gain3.data = network.gain3.data + eta * (1/network.gain3.data - 2*(Euy3/iter_num - Eu3*Ey3/iter_num))
#             network.gain4.data = network.gain4.data + eta * (1/network.gain4.data - 2*(Euy4/iter_num - Eu4*Ey4/iter_num))
#             network.gain5.data = network.gain5.data + eta * (1/network.gain5.data - 2*(Euy5/iter_num - Eu5*Ey5/iter_num))
#             network.gain6.data = network.gain6.data + eta * (1/network.gain6.data - 2*(Euy6/iter_num - Eu6*Ey6/iter_num))
#             network.gain7.data = network.gain7.data + eta * (1/network.gain7.data - 2*(Euy7/iter_num - Eu7*Ey7/iter_num))
#             network.gain8.data = network.gain8.data + eta * (1/network.gain8.data - 2*(Euy8/iter_num - Eu8*Ey8/iter_num))
#             network.gain9.data = network.gain9.data + eta * (1/network.gain9.data - 2*(Euy9/iter_num - Eu9*Ey9/iter_num))
        
            # Variance
            network.gain1.data = network.gain1.data + eta * (1/network.gain1.data - 2*(Euu1/iter_num - Eu1*Eu1/iter_num))
            network.gain2.data = network.gain2.data + eta * (1/network.gain2.data - 2*(Euu2/iter_num - Eu2*Eu2/iter_num))
            network.gain3.data = network.gain3.data + eta * (1/network.gain3.data - 2*(Euu3/iter_num - Eu3*Eu3/iter_num))
            network.gain4.data = network.gain4.data + eta * (1/network.gain4.data - 2*(Euu4/iter_num - Eu4*Eu4/iter_num))
            network.gain5.data = network.gain5.data + eta * (1/network.gain5.data - 2*(Euu5/iter_num - Eu5*Eu5/iter_num))
            network.gain6.data = network.gain6.data + eta * (1/network.gain6.data - 2*(Euu6/iter_num - Eu6*Eu6/iter_num))
            network.gain7.data = network.gain7.data + eta * (1/network.gain7.data - 2*(Euu7/iter_num - Eu7*Eu7/iter_num))
            network.gain8.data = network.gain8.data + eta * (1/network.gain8.data - 2*(Euu8/iter_num - Eu8*Eu8/iter_num))
            network.gain9.data = network.gain9.data + eta * (1/network.gain9.data - 2*(Euu9/iter_num - Eu9*Eu9/iter_num))
            
            network.bias1.data = network.bias1.data + eta * (-2 * Ey1/iter_num)
            network.bias2.data = network.bias2.data + eta * (-2 * Ey2/iter_num)
            network.bias3.data = network.bias3.data + eta * (-2 * Ey3/iter_num)
            network.bias4.data = network.bias4.data + eta * (-2 * Ey4/iter_num)
            network.bias5.data = network.bias5.data + eta * (-2 * Ey5/iter_num)
            network.bias6.data = network.bias6.data + eta * (-2 * Ey6/iter_num)
            network.bias7.data = network.bias7.data + eta * (-2 * Ey7/iter_num)
            network.bias8.data = network.bias8.data + eta * (-2 * Ey8/iter_num)
            network.bias9.data = network.bias9.data + eta * (-2 * Ey9/iter_num)
            
            eta = 0.996*eta
            
    print("Finished training!\n")

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

seed = 986

# #Train IP Model
# torch.manual_seed(seed)
# IPnet = Net()
# IPnet = IPnet.to(device)

# optimizer1 = optim.Adam(IPnet.parameters(), lr=0.001)
# print("Training IP Net")
# train_model(IPnet, optimizer1, seed, True)

# #Train Standard Model
# torch.manual_seed(seed)
# net = Net()
# net = net.to(device)

# optimizer2 = optim.Adam(net.parameters(), lr=0.001)
# print("Training Standard Net")
# train_model(net, optimizer2, seed, False)

#Train Deep IP Model
torch.manual_seed(seed)
DIPnet = DNet(IP=True)
DIPnet = DIPnet.to(device)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, DIPnet.parameters()), lr=3e-4)
print("Training Deep IP Net")
train_deep_model(DIPnet, optimizer, seed, True)

#Train Deep Standard Model
torch.manual_seed(seed)
Dnet = DNet(IP=False)
Dnet = Dnet.to(device)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, Dnet.parameters()), lr=3e-4)
print("Training Deep Standard Net")
train_deep_model(Dnet, optimizer, seed, False)

# for param in IPnet.parameters():
#     print(param.data)
    
# for param in IPnet.parameters():
#     print(param.data)

# c_net = copy.deepcopy(net)

Training Deep IP Net
[1,   100] loss: 0.088
[1,   200] loss: 0.061
[1,   300] loss: 0.058
[2,   100] loss: 0.055
[2,   200] loss: 0.054
[2,   300] loss: 0.055
[3,   100] loss: 0.054
[3,   200] loss: 0.053
[3,   300] loss: 0.053
[4,   100] loss: 0.052
[4,   200] loss: 0.053
[4,   300] loss: 0.052
[5,   100] loss: 0.051
[5,   200] loss: 0.052
[5,   300] loss: 0.052
[6,   100] loss: 0.052
[6,   200] loss: 0.052
[6,   300] loss: 0.051
[7,   100] loss: 0.050
[7,   200] loss: 0.052
[7,   300] loss: 0.051
[8,   100] loss: 0.051
[8,   200] loss: 0.051
[8,   300] loss: 0.050
[9,   100] loss: 0.050
[9,   200] loss: 0.051
[9,   300] loss: 0.050
[10,   100] loss: 0.050
[10,   200] loss: 0.051
[10,   300] loss: 0.051
[11,   100] loss: 0.051
[11,   200] loss: 0.051
[11,   300] loss: 0.051
[12,   100] loss: 0.050
[12,   200] loss: 0.051
[12,   300] loss: 0.050
[13,   100] loss: 0.050
[13,   200] loss: 0.050
[13,   300] loss: 0.051
[14,   100] loss: 0.050
[14,   200] loss: 0.050
[14,   300] loss: 0.05

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        u1, y1, u2, y2, u3, y3, u4, y4, u5, y5, u6, y6, u7, y7, u8, y8, u9, y9  = DIPnet(images)
        _, predicted = torch.max(y9.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the IP network on the 10000 test images: %d %%' % (
    100 * correct / total))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        u1, y1, u2, y2, u3, y3, u4, y4, u5, y5, u6, y6, u7, y7, u8, y8, u9, y9  = Dnet(images)
        _, predicted = torch.max(y9.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the standard network on the 10000 test images: %d %%' % (
    100 * correct / total))

# val1, ind1 = DIPnet.fc1.weight.max(0)
# max_weight = val1.max(0)
# print(max_weight)

# val1, ind1 = Dnet.fc1.weight.max(0)
# max_weight = val1.max(0)
# print(max_weight)

Accuracy of the IP network on the 10000 test images: 66 %
Accuracy of the standard network on the 10000 test images: 75 %
